# Deepdreaming Multiple Caffe Models + Warping, Optical Flow

### Directions:
1: Choose starting frame, name it frame00000.jpg, place file in framedir = '<i>dirname</i>' defined below. <br>
2: Place goals file in base directory, or modify goals file location and place in framedir (see comments below). Define your parameters in goals file (these can be modified during the run) <br>
3: Run the notebook after setting notebook parameters as well to generate frames. <br>
4: Generate movie with 'ffmpeg -i frame%05d.jpg -vcodec libx264 -r <i>30</i> <i>movie-name.mp4</i>'

In [ ]:
# Import modules
import argparse
import time
import os
import errno
import subprocess
import sys
import matplotlib
matplotlib.use('Agg')
import numpy as np
import scipy.ndimage as nd
import scipy
import PIL.Image, PIL.ImageEnhance
import caffe
from cStringIO import StringIO
from google.protobuf import text_format
from IPython.display import clear_output, Image, display
from skimage import transform, io, img_as_float, img_as_ubyte
from resizeimage import resizeimage

caffe.set_mode_gpu()    # Comment out for CPU only
caffe.set_device(0)     # Comment out for CPU only

<b>Set</b> desired parameters:

In [ ]:
guided = "True"            # "True" or "False"
blendoriginal = "True"     # Blend in the original frame? "True" or "False"
bluramount = 0.3           # smaller = favors dream over original image during blending  ... 0 - 1
s = 0.003                  # Zoom speed. Positive for inward / Negative for outward.
width = 1024               # Frame width
height = 683               # Frame height
centerwidth = width / 2    # Center whirlpool based on chosen width
centerheight = height / 2  # Center whirlpool based on chosen height
octave_n=4                 # Otherwise defined in Goals file located in %framedir
octave_scale=2.1           # Otherwise defined in Goals file located in %framedir
clip=True  
it = 1                     # n frames to blend between dreams
mod = "googlenet" 
framedir = 'frames'        # Output frame directory
frame_i = 0                # Current frame
!rm -rf tmp && mkdir tmp   # Clear and Create tmp directory
           # Model 1 path + files
model_path_loc = '/usr/lib/python2.7/site-packages/caffe/models/bvlc_googlenet/'
net_fn_loc     = 'deploy.prototxt'
param_fn_loc   = 'bvlc_googlenet.caffemodel'
           # Model 2 path + files
model_path_loc = '/usr/lib/python2.7/site-packages/caffe/models/googlenet_places205/'
net_fn_loc     = 'deploy_places205.protxt'
param_fn_loc   = 'googlelet_places205_train_iter_2400000.caffemodel'

<b>Run</b> the following to set up before the dream block:

In [ ]:
infile = "frame%05d.jpg"%frame_i                             # How to reach the current frame
original = PIL.Image.open("%s/%s"%(framedir,infile))         # Capture First frame from framedir for blending purposes
PIL.Image.fromarray(np.uint8(original)).save("tmp/original.jpg") # Save first frame for blending purposes

logfile="log"                                                # Define logfile name / location.  logfile="%s/log"%framedir for log in framedir
log = open(logfile, 'a')                                     # Logging functions below
log.write("\n\nrestart %s\n\n"%sys.argv[0])
log.flush()

In [ ]:
# Define the Definitions!

def showarray(a, fmt='jpeg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

def get_data():
    goals = open("goals", 'r')    # goals = open("%s/goals"%framedir, 'r')   to place goals in framedir
    line = goals.readline()
    line.strip()
    [t0, t1, t2, t3, eb, ec, es] = line.split()
    v1 = [float(t0), float(t1), float(t2), float(t3), float(eb), float(ec), float(es)]
    done = False
    v2 = []
    while not done:
        line = goals.readline()
        if line:
            if line[0]=='#':
                continue
            else:
                dat = line.split()
                if len(dat)== 5:
                    [a,b,c,d,e] = dat
                    v2.append([a,b,int(c),int(d),float(e)])
        else:
            done=True
    for j in v2:
        goals.close()
    return [v1,v2]

def getnet(mod):
    if mod == "googlenet":
        model_path = model_path_loc
        net_fn   = model_path + net_fn_loc
        param_fn = model_path + param_fn_loc
    elif mod == "googlenet_places205":
        model_path = model_path_loc
        net_fn   = model_path + net_fn_loc
        param_fn = model_path + param_fn_loc
    else:
        abort("model not found")
    model = caffe.io.caffe_pb2.NetParameter()
    text_format.Merge(open(net_fn).read(), model)
    model.force_backward = True
    open('tmp.prototxt', 'w').write(str(model))
    return caffe.Classifier('tmp.prototxt', param_fn,
                            mean = np.float32([104.0, 116.0, 122.0]), 
                            channel_swap = (2,1,0)) 

def preprocess(net, img):
    return np.float32(np.rollaxis(img, 2)[::-1]) - net.transformer.mean['data']
def deprocess(net, img):
    return np.dstack((img + net.transformer.mean['data'])[::-1])

def objective_L2(dst):
    dst.diff[:] = dst.data 

def make_step(net, step_size=1.5, end='inception_4c/output', jitter=32, clip=True, objective=objective_L2):
    '''Basic gradient ascent step.'''
    src = net.blobs['data'] 
    dst = net.blobs[end]
    ox, oy = np.random.randint(-jitter, jitter+1, 2)
    src.data[0] = np.roll(np.roll(src.data[0], ox, -1), oy, -2)
    net.forward(end=end)
    objective(dst)  
    net.backward(start=end)
    g = src.diff[0]
    src.data[:] += step_size/np.abs(g).mean() * g
    src.data[0] = np.roll(np.roll(src.data[0], -ox, -1), -oy, -2)    
    if clip:
        bias = net.transformer.mean['data']
        src.data[:] = np.clip(src.data, -bias, 255-bias)

def deepdream(net, base_img, iter_n=10, octave_n=4, octave_scale=1.4, end="inception_4c/output", clip=True, **step_params):
    octaves = [preprocess(net, base_img)]
    for i in xrange(octave_n-1):
        octaves.append(nd.zoom(octaves[-1], (1, 1.0/octave_scale,1.0/octave_scale), order=1))
    src = net.blobs['data']
    detail = np.zeros_like(octaves[-1]) 
    for octave, octave_base in enumerate(octaves[::-1]):
        h, w = octave_base.shape[-2:]
        if octave > 0:
            h1, w1 = detail.shape[-2:]
            detail = nd.zoom(detail, (1, 1.0*h/h1,1.0*w/w1), order=1)
        src.reshape(1,3,h,w) 
        src.data[0] = octave_base+detail
        for i in xrange(iter_n):
            make_step(net, end=end, clip=clip, **step_params)
            vis = deprocess(net, src.data[0])
            if not clip: 
                vis = vis*(255.0/np.percentile(vis, 99.98))
        detail = src.data[0]-octave_base
    return deprocess(net, src.data[0])

def fisheye(xy):
    center = [centerwidth, centerheight]
    xc, yc = (xy - center).T
    r = np.sqrt(xc**2 + yc**2)
    theta = np.arctan2(yc, xc)
    rb = r*(1+np.exp(-r/t1)/t0)
    shift = (np.exp(-(r/(t3))))*np.pi/t2
    ret = np.column_stack((rb * np.cos(theta+shift), rb * np.sin(theta+shift))) + center
    return ret
    
def objective_guide(dst):  
    x = dst.data[0].copy()
    y = guide_features
    ch = x.shape[0]
    x = x.reshape(ch,-1)
    y = y.reshape(ch,-1)
    A = x.T.dot(y)
    dst.diff[0].reshape(ch,-1)[:] = y[:,A.argmax(1)]

# Test data retrieval from goals file. If error, then define goals file in framedir
print get_data()
[[t0, t1, t2, t3, eb, ec, es], dd] = get_data()  

<b>Define</b> guide images to loop through per frame if guided = "True":

In [ ]:
genguide = PIL.Image.open("tmp/original.jpg")
getguide = resizeimage.resize_width(genguide, 250)
PIL.Image.fromarray(np.uint8(getguide)).save("tmp/newguide.jpg") 

#guide_loop = ['tmp/newguide.jpg']  # Guide based on previous frame
guide_loop = ['guideimages/1.png', 'guideimages/2.png', 'guideimages/3.png', 'guideimages/4.png', 'guideimages/5.png', 'guideimages/6.png', 'guideimages/7.png', 'guideimages/8.png', 'guideimages/9.png', 'guideimages/10.png', 'guideimages/11.png', 'guideimages/12.png', 'guideimages/13.png', 'guideimages/14.png', 'guideimages/15.png', 'guideimages/16.png', 'guideimages/17.png', 'guideimages/18.png', 'guideimages/19.png', 'guideimages/20.png', 'guideimages/21.png', 'guideimages/22.png', 'guideimages/23.png', 'guideimages/24.png', 'guideimages/25.png', 'guideimages/26.png', 'guideimages/27.png', 'guideimages/28.png', 'guideimages/29.png', 'guideimages/30.png', 'guideimages/31.png', 'guideimages/32.png', 'guideimages/33.png', 'guideimages/34.png', 'guideimages/35.png', 'guideimages/36.png', 'guideimages/37.png', 'guideimages/38.png', 'guideimages/39.png', 'guideimages/40.png', 'guideimages/41.png', 'guideimages/42.png', 'guideimages/43.png', 'guideimages/44.png', 'guideimages/45.png', 'guideimages/46.png', 'guideimages/47.png', 'guideimages/48.png', 'guideimages/49.png', 'guideimages/50.png', 'guideimages/51.png', 'guideimages/52.png', 'guideimages/53.png', 'guideimages/54.png', 'guideimages/55.png', 'guideimages/56.png', 'guideimages/57.png', 'guideimages/58.png', 'guideimages/59.png', 'guideimages/60.png', 'guideimages/61.png', 'guideimages/62.png', 'guideimages/63.png', 'guideimages/64.png', 'guideimages/65.png', 'guideimages/66.png', 'guideimages/67.png', 'guideimages/68.png', 'guideimages/69.png', 'guideimages/70.png', 'guideimages/71.png', 'guideimages/72.png', 'guideimages/73.png', 'guideimages/74.png', 'guideimages/75.png', 'guideimages/76.png', 'guideimages/77.png', 'guideimages/78.png', 'guideimages/79.png', 'guideimages/80.png', 'guideimages/81.png', 'guideimages/82.png', 'guideimages/83.png', 'guideimages/84.png', 'guideimages/85.png', 'guideimages/86.png', 'guideimages/87.png', 'guideimages/88.png', 'guideimages/89.png', 'guideimages/90.png', 'guideimages/91.png', 'guideimages/92.png', 'guideimages/93.png', 'guideimages/94.png', 'guideimages/95.png', 'guideimages/96.png', 'guideimages/97.png', 'guideimages/98.png', 'guideimages/99.png', 'guideimages/100.png', 'guideimages/101.png', 'guideimages/102.png', 'guideimages/103.png', 'guideimages/104.png', 'guideimages/105.png', 'guideimages/106.png', 'guideimages/107.png', 'guideimages/108.png', 'guideimages/109.png', 'guideimages/110.png', 'guideimages/111.png', 'guideimages/112.png', 'guideimages/113.png', 'guideimages/114.png', 'guideimages/115.png', 'guideimages/116.png', 'guideimages/117.png', 'guideimages/118.png', 'guideimages/119.png', 'guideimages/120.png', 'guideimages/121.png', 'guideimages/122.png', 'guideimages/123.png', 'guideimages/124.png', 'guideimages/125.png', 'guideimages/126.png', 'guideimages/127.png', 'guideimages/128.png', 'guideimages/129.png', 'guideimages/130.png', 'guideimages/131.png', 'guideimages/132.png', 'guideimages/133.png', 'guideimages/134.png', 'guideimages/135.png', 'guideimages/136.png', 'guideimages/137.png', 'guideimages/138.png', 'guideimages/139.png', 'guideimages/140.png', 'guideimages/141.png', 'guideimages/142.png', 'guideimages/143.png', 'guideimages/144.png', 'guideimages/145.png', 'guideimages/146.png', 'guideimages/147.png', 'guideimages/148.png', 'guideimages/149.png', 'guideimages/150.png', 'guideimages/151.png', 'guideimages/152.png', 'guideimages/153.png', 'guideimages/154.png', 'guideimages/155.png', 'guideimages/156.png', 'guideimages/157.png', 'guideimages/158.png', 'guideimages/159.png', 'guideimages/160.png', 'guideimages/161.png', 'guideimages/162.png', 'guideimages/163.png', 'guideimages/164.png', 'guideimages/165.png', 'guideimages/166.png', 'guideimages/167.png', 'guideimages/168.png', 'guideimages/169.png', 'guideimages/170.png', 'guideimages/171.png', 'guideimages/172.png', 'guideimages/173.png', 'guideimages/174.png', 'guideimages/175.png', 'guideimages/176.png', 'guideimages/177.png', 'guideimages/178.png', 'guideimages/179.png', 'guideimages/180.png', 'guideimages/181.png', 'guideimages/182.png', 'guideimages/183.png', 'guideimages/184.png', 'guideimages/185.png', 'guideimages/186.png', 'guideimages/187.png', 'guideimages/188.png', 'guideimages/189.png', 'guideimages/190.png', 'guideimages/191.png', 'guideimages/192.png', 'guideimages/193.png', 'guideimages/194.png', 'guideimages/195.png', 'guideimages/196.png', 'guideimages/197.png', 'guideimages/198.png', 'guideimages/199.png', 'guideimages/200.png', 'guideimages/201.png', 'guideimages/202.png', 'guideimages/203.png', 'guideimages/204.png', 'guideimages/205.png', 'guideimages/206.png', 'guideimages/207.png', 'guideimages/208.png', 'guideimages/209.png', 'guideimages/210.png']
guide = np.float32(PIL.Image.open(guide_loop[frame_i % len (guide_loop)]))  # Loop through guide images determined by current frame_i number.
print guide_loop[frame_i % len (guide_loop)]                                # Show currently selected guide image based on current frame_i number.
showarray(guide)                                                            # Shows guide image selected for the frame number: frame_i

#### Dream Time:

In [ ]:
new = io.imread("%s/%s"%(framedir,infile))
frame_i += 1
old = new

net = getnet("googlenet")
net1 = getnet("googlenet_places205")
new = old

h, w = (height, width)
box = (0, 0, width, height)

for i in xrange(10000):
    new = nd.affine_transform(new, [1-s,1-s,1], [h*s/2,w*s/2,0], order=1)
    PIL.Image.fromarray(np.uint8(new)).save("tmp/temp.jpg")  
    newguide = PIL.Image.open("tmp/temp.jpg")
    newg = resizeimage.resize_width(newguide, 250)
    PIL.Image.fromarray(np.uint8(newg)).save("tmp/newguide.jpg")  
    if blendoriginal == "True":
        new = PIL.Image.blend(PIL.Image.fromarray(np.uint8(new)), PIL.Image.open("tmp/original.jpg"), bluramount)
    new = img_as_ubyte(new)
    new = transform.warp(new, fisheye)
    new = img_as_ubyte(new)
    old = img_as_ubyte(old)
    old = transform.warp(old, fisheye)
    old = img_as_ubyte(old)
    nim = PIL.Image.fromarray(np.uint8(new))
    oim = PIL.Image.fromarray(np.uint8(old))
    bp = (frame_i%it+0.)/(it)
    out = PIL.Image.blend(oim, nim, bp)
    out.save("%s/frame%05d.jpg"%(framedir, frame_i))
    frame_i += 1
    if frame_i%it==0:
        log.write("\nframe%05d:\n"%frame_i)
        [[t0, t1, t2, t3, eb, ec, es], dd] = get_data()
        log.write("%s %s %s %s %s %s %s\n"%(t0,t1,t2,t3,eb,ec,es))
        old = new
        for [end, mod, iter_n, octave_n, octave_scale] in dd:
            log.write("%s %s %s %s %s\n"%(end, mod, iter_n, octave_n, octave_scale))
            log.flush()
            if guided == "False":
                if mod == "googlenet":
                    new = deepdream(net, new, end=end, iter_n=iter_n, octave_n=octave_n, octave_scale=octave_scale)
                elif mod == "googlenet_places205":
                    new = deepdream(net1, new, end=end, iter_n=iter_n, octave_n=octave_n, octave_scale=octave_scale)
                elif mod == "both":
                    new = deepdream(net, new, end=end, iter_n=iter_n, octave_n=octave_n, octave_scale=octave_scale)
                    new = deepdream(net1, new, end=end, iter_n=iter_n, octave_n=octave_n, octave_scale=octave_scale)
                else:
                    print "error: %s not recognized"%mod
                    log.write("error: %s not recognized\n"%mod)
            elif guided == "True":
                if mod == "googlenet":
                    h, w = guide.shape[:2]
                    src, dst = net.blobs['data'], net.blobs[end]
                    src.reshape(1,3,h,w)
                    src.data[0] = preprocess(net, guide)
                    net.forward(end=end)
                    guide_features = dst.data[0].copy()
                    new = deepdream(net, new, end=end, iter_n=iter_n, objective=objective_guide, octave_n=octave_n, octave_scale=octave_scale)
                elif mod == "googlenet_places205":
                    h, w = guide.shape[:2]
                    src, dst = net.blobs['data'], net.blobs[end]
                    src.reshape(1,3,h,w)
                    src.data[0] = preprocess(net, guide)
                    net.forward(end=end)
                    guide_features = dst.data[0].copy()
                    new = deepdream(net1, new, end=end, iter_n=iter_n, objective=objective_guide, octave_n=octave_n, octave_scale=octave_scale)
                elif mod == "both":
                    h, w = guide.shape[:2]
                    src, dst = net.blobs['data'], net.blobs[end]
                    src.reshape(1,3,h,w)
                    src.data[0] = preprocess(net, guide)
                    net.forward(end=end)
                    guide_features = dst.data[0].copy()
                    new = deepdream(net, new, end=end, iter_n=iter_n, objective=objective_guide, octave_n=octave_n, octave_scale=octave_scale)
                    new = deepdream(net1, new, end=end, iter_n=iter_n, objective=objective_guide, octave_n=octave_n, octave_scale=octave_scale)
                else:
                    print "error: %s not recognized"%mod
                    log.write("error: %s not recognized\n"%mod)
            else:
                print "set guided = 'True' or... guided = 'False'"
                log.write("error: guide = 'True' or 'False' not set")
        new = PIL.Image.fromarray(np.uint8(new))
        new = PIL.ImageEnhance.Brightness(new).enhance(eb)
        new = PIL.ImageEnhance.Contrast(new).enhance(ec)
        new = img_as_ubyte(new)

<b>Generate</b> your video:

In [ ]:
!ffmpeg -i frames/frame%05d.jpg -vcodec libx264 -r 60 dream.mp4